# DSCI-508 Project 7
### Matt Snyder

In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk

# To start, we need some text to play with. NLTK has many corpora and resources for you to explore natural language. 
# A one-off run of nltk.download() will get you all the resources in one go. Once you've done that you should have 
# a repository of interesting texts including stuff like Moby Dick and an Inaugural Address Corpus

nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/matt/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [3]:
from nltk.corpus import movie_reviews
print(movie_reviews.categories()) # 'pos' (positive) and 'neg' (negative)
# print(movie_reviews.fileids()) # Lists review filenames


['neg', 'pos']


In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from string import punctuation
customStopWords=set(stopwords.words('english')+list(punctuation))

In [34]:
from nltk.probability import FreqDist
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import math

## Create DataFrame of Positive Movie Reviews

In [7]:
pos_fileids = movie_reviews.fileids(categories='pos')

In [8]:
movie_reviews.raw(fileids=pos_fileids[0])

'films adapted from comic books have had plenty of success , whether they\'re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there\'s never really been a comic book like from hell before . \nfor starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid \'80s with a 12-part series called the watchmen . \nto say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . \nthe book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . \nin other words , don\'t dismiss this film because of its source . \nif you can get past the whole comic book thing , you might find another stumbling block in from hell\'s directors , albert and allen hughes . \ngetting the hughes brothers to direct this seem

In [9]:
# make a dataframe with two columns: text of the movie review, tokenized words (unordered) minus stop words
# also a column for the tf/idf vector
df_dict = {'text':[], 'words':[], 'vector':[]}
for id in pos_fileids:
    df_dict['text'].append (movie_reviews.raw(fileids=id))
    df_dict['words'].append (list(movie_reviews.words(fileids=id)))
    df_dict['vector'].append ([])
df = pd.DataFrame(df_dict)
# remove stopwords and punctuation
df['words'] = df['words'].apply(lambda x: [term for term in x if term not in customStopWords])
df.head()

,text,words,vector
0,films adapted from comic books have had plenty...,"[films, adapted, comic, books, plenty, success...",[]
1,every now and then a movie comes along from a ...,"[every, movie, comes, along, suspect, studio, ...",[]
2,you've got mail works alot better than it dese...,"[got, mail, works, alot, better, deserves, ord...",[]
3,""" jaws "" is a rare film that grabs your atten...","[jaws, rare, film, grabs, attention, shows, si...",[]
4,moviemaking is a lot like being the general ma...,"[moviemaking, lot, like, general, manager, nfl...",[]


## Compute Vectors using TF/IDF

In [10]:
# master vector of union of all words in reviews
master_set = set(df.loc[0, 'words'])
for i in range(1, len(df)):
    master_set.update(df.loc[i, 'words']) # accumulate union of words in set
master_array = np.array(list(master_set)) # convert set to array
print (len(master_array))

30236


In [38]:
# compute tf/idf for each review

# first pass, assemble vectors with just term freq
tf_vectors = []
for i in range(0, len(df)):
    fdist = FreqDist(df.loc[i, 'words'])
    vector = np.array([fdist[word] for word in master_array])
    tf_vectors.append(vector)

In [40]:
# using vectors of tf, compute document counts of these same words

# convert non-zero term counts into 1's
def zero_or_not(x):
    return 1 if x > 0 else 0
zero_or_not_vectorized = np.vectorize(zero_or_not)
doc_count_vectors = [zero_or_not_vectorized(v) for v in tf_vectors]

# convert array list to matrix and sum the 1's 
doc_count_matrix = np.stack(doc_count_vectors)
doc_counts_per_word = np.sum(doc_count_matrix, axis=0)
print (doc_counts_per_word[0:20])

[  1   2   1  92   2   5   5   1   5  13  17   1  14 148   2  11   1   1
   2  16]


In [41]:
# compute idf = inverse document frequency
# = log of the number of documents divided by the log of one plus the number of documents containing that word
def idf(doc_count:int, total_docs:int):
    return math.log( total_docs / 1+doc_count )
idf_vectorized = np.vectorize(idf)
total_docs = len(df)
idf_per_word = idf_vectorized(doc_counts_per_word, total_docs)
print (idf_per_word[0:20])

[6.90875478 6.90975328 6.90875478 6.99576616 6.90975328 6.91274282
 6.91274282 6.90875478 6.91274282 6.9206715  6.9246124  6.90875478
 6.92165818 7.04577658 6.90975328 6.91869522 6.90875478 6.90875478
 6.90975328 6.92362863]


In [46]:
# product of tf vectors and idf vector gives final tf/idf vectors
tf_idf_vectors = [v * idf_per_word for v in tf_vectors]
print (tf_idf_vectors[0][0:20])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         7.04577658 0.         0.         0.         0.
 0.         0.        ]


In [48]:
# add tf/idf vector column to dataframe
df['vector'] = tf_idf_vectors
print (df.loc[0, 'vector'][0:20])
df.head()

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         7.04577658 0.         0.         0.         0.
 0.         0.        ]


,text,words,vector,cluster
0,films adapted from comic books have had plenty...,"[films, adapted, comic, books, plenty, success...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",82
1,every now and then a movie comes along from a ...,"[every, movie, comes, along, suspect, studio, ...","[0.0, 0.0, 0.0, 6.9957661563048505, 0.0, 0.0, ...",82
2,you've got mail works alot better than it dese...,"[got, mail, works, alot, better, deserves, ord...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",39
3,""" jaws "" is a rare film that grabs your atten...","[jaws, rare, film, grabs, attention, shows, si...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",32
4,moviemaking is a lot like being the general ma...,"[moviemaking, lot, like, general, manager, nfl...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",39


## Test Vectors With Cluster Model

In [49]:
# try making a cluster model with large number of clusters
X = np.stack(df['vector'].values)
print (X.shape)
model = KMeans(n_clusters=100)
model.fit(X)

(1000, 30236)


KMeans(n_clusters=100)

In [50]:
# add cluster labels onto dataframe
clusters = model.labels_ # labels of all the X data; no need to run predict to get it, since it's saved in model
df['cluster'] = clusters
df.head()

,text,words,vector,cluster
0,films adapted from comic books have had plenty...,"[films, adapted, comic, books, plenty, success...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",77
1,every now and then a movie comes along from a ...,"[every, movie, comes, along, suspect, studio, ...","[0.0, 0.0, 0.0, 6.9957661563048505, 0.0, 0.0, ...",77
2,you've got mail works alot better than it dese...,"[got, mail, works, alot, better, deserves, ord...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",77
3,""" jaws "" is a rare film that grabs your atten...","[jaws, rare, film, grabs, attention, shows, si...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",26
4,moviemaking is a lot like being the general ma...,"[moviemaking, lot, like, general, manager, nfl...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",70


In [51]:
# find small clusters, i.e. those containing 2 reviews
hist_counts, bin_edges = np.histogram(clusters, bins=100)
clusters_with_two_reviews = [i for i, count in enumerate(hist_counts) if count == 2]
print (clusters_with_two_reviews)

[5, 7, 12, 29, 58, 68, 69, 87, 90, 96, 97]


In [52]:
# show the reviews from the 2-review clusters
for cluster in clusters_with_two_reviews:
    sel = df.loc[:,'cluster'] == cluster
    similar_reviews_df = df.loc[sel, :]
    review_texts = similar_reviews_df['text'].values
    for i in range(0, len(review_texts)):
        print (review_texts[i][0:500])
        print ()
    print ('----------------------------------------------')

 " seven " is one of the best mystery movies i've ever seen . 
it's extremely intriguing and suspenseful , but it's also quite fun . 
it's a serial killer mystery , but you don't care so much about making the killer pay as much as hope they just catch the killer . 
the story is a cops-on-the-trail-of-serial-killer mystery . 
someone is murdering people who are offenders of " the seven deadly sins . " 
a fat man ( gluttony ) was forced to eat himself to death , a lawyer ( greed ) is slaughtered b

one of the more unusual and suggestively violent films ever made , " se7en " is just a few steps away from brilliance . 
however , those few steps away are only a few steps from a level of evil and depravity that few people want to discuss . 
the film received mixed reviews upon release , but there's an underlying sinister appeal and intrigue to this that many critics perhaps did not notice . 
at times it gets preachy about the horrors it seems to be criticizing , but the script and the ultima

## Conclusion
The reviews in the 2-review clusters do indeed use similar words, and many are about the same movie, therefore TF/IDF can be used to compare two chunks of text.  And many of the results in the 2-review clusters were apparently duplicates, so this is also a useful method for finding duplicates

In [25]:
print (vector_matrix)
print (vector_matrix.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(1000, 30236)


In [26]:
maxes = np.sum(vector_matrix, axis=0)
print (maxes[0:100])

[ 1  1  1  2  1  1  4  1  3  1  3  1  1 10  1  4  1  1  2  2  1  1  4  1
  1  1  1  1  3  1  1  1  1  1  1  1  1  1  1  1  1  3  2  1  1  1  2  3
  1  1  1  1  1  1  9  1  1  1  1  1  1  1  1  1  1  1  1  1  2  6  2  2
  1  2  1  1  2  1  1  1  2  2  1  4  3  2  1  3  1  1  1  1  1  1  6 19
  1  2  1  1]


In [29]:
def zero_or_not(x):
    return 1 if x > 0 else 0
zero_or_not_vectorized = np.vectorize(zero_or_not)
test = zero_or_not_vectorized(vectors[0])
print (test[0:100])

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
